In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
nltk.download("stopwords")
nltk.download("punkt")

import tweepy
from kafka import KafkaProducer
from time import sleep
import json 
import pandas as pd

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\chaima\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chaima\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chaima\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def preprocessing(text, lang): #removing useless information in the tweets that won't serve in sentiment analysis
    text = text.lower()
    text = re.sub("https?://[\S]+", "", text) #remove links
    text = re.sub(r"@[\S]+", "", text) #remove tags
    text = re.sub("[^a-zA-Z]", " ", text) #remove numbers 
    text = word_tokenize(text) #tokenize the text
    text = [word for word in text if word not in stopwords.words(lang)] #remove stop words
    text = " ".join(word for word in text)
    return text

In [3]:
column_names=["lang","possibly_sensitive",'text', 'created_at', 'label']
df = pd.DataFrame(columns = column_names)
counter = 1 # to debug the message number
topic_name = 'Data_Stream_Project'
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAALiNlAEAAAAA1cWghqMR6ix%2BO4BM5%2BIHjQBAcj4%3DfOAtXOgmF57g09Wp0jrDuX5WCS1xXhdSRx7lLmyhyYfHQivCld')
# searching tweets about covid
query = 'ukraine'
producer = KafkaProducer(bootstrap_servers="localhost:9092", value_serializer=lambda x: json.dumps(x).encode("utf8"),)

print(producer.bootstrap_connected()) 

True


In [4]:
while True:
    for tweet in tweepy.Paginator(client.search_recent_tweets, query=query,
    tweet_fields=[ 'created_at', 'lang', 'possibly_sensitive'], max_results=30).flatten():
        if tweet["lang"]=='en':
            lang='english'
        elif tweet["lang"]=='fr':
            lang='french'
        msg= preprocessing(tweet['text'], lang)    
        y=0 #neutral by default
        score = SentimentIntensityAnalyzer().polarity_scores(msg)
        #labelling data:
        if score['neg'] > score['pos']:
            y = -1 #-1 means negative
        elif score['pos'] > score['neg']:
            y= 1 #+1 means positive
            
        message = {
                "lang": tweet["lang"],
                "possibly_sensitive": tweet["possibly_sensitive"],
                "text": msg,
                "created_at": tweet["created_at"].strftime("%Y-%m-%d %H:%M:%S"),
                "label" : y,
            }
        print(message['label'])
        if (tweet["lang"]=='en') or (tweet["lang"]=='fr'):
            df = df.append(message, ignore_index=True)
        print(tweet, tweet['lang'], tweet['possibly_sensitive'])
        print('\n')
        producer.send(topic_name,message)
        print('Sending message',counter)
        counter+=1
        
    sleep(1)

0
@Diomedes2022 @MelnykAndrij @Bundeskanzler That package would cost problably more than Ukraine's GDP. en False


Sending message 1
-1
RT @RebHarms: Syrian’s solidarity with Ukrainians is strong. For horrid reasons. 

Russia‘s impunity for the terrible war crimes in Ukraine… en False


Sending message 2
0
RT @OuchikhKarim: "L'Apothéose de la guerre" est un tableau du peintre russe Vassili Verechtchaguine réalisé en 1871. 

Sur son encadrement… fr False


Sending message 3
1
@FriendlyOrendi @davidkurten But it's Ukrainians dying in far greater numbers. Where is the fantasy inventive.

Stated goal - demilitarisation of Ukraine.

It's happening, why stop before it's done. en False


Sending message 4
0
RT @Jona_Mar78: Wielka Brytania wyśle na Ukrainę 4 śmigłowce szturmowe Apache AH64 E, uzbrojone w pociski Hellfire, informuje brytyjska pub… pl False


Sending message 5
-1
RT @KyivIndependent: The death toll of the Jan. 14 Russian cruise missile strike on an apartment building in Dnipro h

<ipython-input-4-be891330dc68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: In a future version, object-dtype columns with all-bool va

Sending message 12
-1
@sishadowdeath @thisisayakasan We Iranian revolutionaries request the world to expel the terrorist ambassadors of the Islamic Republic. By doing this, you will save the children of Iran, Ukraine, Iraq and Syria. Be the voice of humanity.
#IRGCterrorists 
#IranRevoIution
#MahsaAmini en False


Sending message 13
-1
@jdomerchet @Corse_Aire @ArnaudDubien @EmmanuelMacron @alexjubelin C'est un enjeux politique pour les usa et vous êtes parfaitement informé à ce sujet. 
La France et même l'Europe n'a rien a tirer de cette guerre et tout a perdre. Alors le états majors fr ont parfaitement raison de refuser les envois d'armes dans ce pays pourri qu'est l'Ukraine fr False


Sending message 14
-1
RT @sternenko: Russians launched a rocket attack on a residential building in Dnipro. 

There are many people under the rubble. This is hel… en False


Sending message 15
-1
RT @lesiavasylenko: 20 killed. 40 missing. 73 wounded. Tragic statistics updated from #Russia attack on #Dni

<ipython-input-4-be891330dc68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: The frame.append m

Sending message 22
1
RT @LakeBerz: Kindness on Ukraine's frontline. @VladaPolishchuk #Dogs #Ukraine #Bakhmut #StandWithUkraine https://t.co/TJQvOQ6uQv https://t… en False


Sending message 23
1
RT @UAWeapons: #Ukraine: ❗ A very notable development - new Ukrainian-made artillery ammunition started to reach the frontlines.

Here we c… en False


Sending message 24
-1
NATO chief hints at more heavy weapons deliveries to Ukraine | NATO News https://t.co/InR42Mx1ri en False


Sending message 25
-1
RT @Pontifex: Brothers and sisters, let us not forget the battered people of Ukraine who are suffering so much! Let us stay close to them w… en False


Sending message 26
1
@berlin_bridge You are brave and beautiful but misguided, Madam Berlin. Russia is known in the world as champion of good relations, stability and peace. War in Ukraine and the inherent atrocities is fuelled by true sponsors of terrorism, the warmongers in the West. en False


Sending message 27
-1
2/ dělostřelecké jednotky. .
 

...

-1
RT @rising_serpent: So you're telling me that Joe Biden was stealing highly classified documents on Ukraine and stashing them in his garage… en False


Sending message 1058
-1
RT @dszeligowski: [SIX] Furthermore, he manipulates by calling certain political circles pro-war or pro-peace ones. Leaving Ukraine alone t… en False


Sending message 1059
1
@newalamcat @CNN So, the "Nazis" is Russia are bad! But the ones in Ukraine, Baltics, Western Europe, in the USA and even in Brazil (they are concentrated in South Brazil) are the good ones? 🫣🤭🤭🤭🤭👏👏 Double standards again! Wow!!!!!🤦🥴 en False


Sending message 1060
-1
RT @Trollstoy88: It took "30 trains, 20 ships and 81 C-17 flights," to move 24 Apache helicopters from Germany to near the battlefield wher… en False


Sending message 1061
0
RT @NOELreports: NATO Secretary General Stoltenberg said that the normalization of relations between Western countries and Russia will not… en False


Sending message 1062
0
RT @_Denyshchenko: Сєвєродон

<ipython-input-4-be891330dc68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: The frame.append m

False


Sending message 1067
0
RT @Maks_NAFO_FELLA: "The German manufacturer of tanks will not be able to deliver them to Ukraine until 2024, even if the government appro… en False


Sending message 1068
-1
RT @AwivaNews: @ConStelz Nicht die #Ukraine führt den Krieg.
Es ist die #NATO.
Nicht um die 🇺🇦 wird Krieg geführt.
Die #USA greifen nach #R… de False


Sending message 1069
1
RT @Tendar: Russian troops in Makiivka were having a party for New Years Eve. This is one of their last pictures before HIMARS missiles wip… en False


Sending message 1070
1
RT @visegrad24: Polish PM Morawiecki is traveling to Berlin tomorrow in an attempt to convince Scholz to send Leopard 2 tanks to Ukraine.… en False


Sending message 1071
1
@Patrick28877491 @thehill Had we done nothing here Russia would have taken Ukraine in days or weeks, many 100's of thousands of ppl on BOTH sides would still be alive and Ukraine would once again be apart of Russia as it has been for most of modern history.

https://t.c

<ipython-input-4-be891330dc68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append(message, ignore_index=True)
<ipython-input-4-be891330dc68>:26: FutureWarning: The frame.append m

KeyboardInterrupt: 

In [5]:
df[df['lang']=='fr']

,lang,possibly_sensitive,text,created_at,label
2,fr,False,rt apoth ose guerre tableau peintre russe vass...,2023-01-15 21:01:27,0
12,fr,False,enjeux politique usa parfaitement inform sujet...,2023-01-15 21:01:26,-1
40,fr,False,chances sauver gens maintenant minimes espoirs...,2023-01-15 21:01:23,1
43,fr,False,rt documents classifi trouv chez joe biden a u...,2023-01-15 21:01:22,0
56,fr,False,rt ukraine a condamn quel conflit afrique fout...,2023-01-15 21:01:20,0
...,...,...,...,...,...
869,fr,False,rt annonce livraison ukraine chars combat lour...,2023-01-15 20:59:19,-1
872,fr,False,va ukraine attends quoi,2023-01-15 20:59:19,0
884,fr,False,rt pendant dias racontent russie crise ukraine...,2023-01-15 20:59:17,0
886,fr,False,rt russie ukraine secr taire g ral otan demand...,2023-01-15 20:59:17,0


In [ ]:
df.to_csv(index=False)

In [6]:
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
df.to_csv('data.zip', index=False,
          compression=compression_opts)

In [7]:
10%100

10

In [8]:
20%100

20